In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_data = pd.read_csv("../input/mnist-in-csv/mnist_train.csv") #reading the csv files using pandas
test_data = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")

In [ ]:
train_data.shape # print the dimension or shape of train data
         
test_data.shape # print the dimension or shape of test data
         

In [ ]:
sns.countplot(train_data["label"])

In [ ]:
# Plotting one sample as well as converting into matrix
one = train_data.iloc[3, 1:]
one.shape
one = one.values.reshape(28,28)
plt.imshow(one, cmap='gray')
plt.title("Digit 1")

In [ ]:
## Separating the X and Y variable
y = train_data['label'].values.astype('int32')
## Dropping the variable 'label' from X variable
X = train_data.drop(columns = 'label').values.astype('float32')
## Printing the size of data
print("X:", X.shape)

In [ ]:
## Normalization
X = X/255.0

In [ ]:
# scaling the features
from sklearn.preprocessing import scale
X_scaled = scale(X)

In [ ]:
from keras.utils import to_categorical
#y_cat = to_categorical(y)
# train test split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size = 0.2, train_size = 0.8, random_state = 10)

In [ ]:
model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)

In [ ]:
# predict
y_pred = model_linear.predict(X_val)

In [ ]:
# confusion matrix and accuracy
from sklearn import metrics
from sklearn.metrics import confusion_matrix
#accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_val, y_pred=y_pred), "\n")
# cm
print(metrics.confusion_matrix(y_true=y_val, y_pred=y_pred))

In [ ]:
# non-linear model
# using rbf kernel, C=1, default value of gamma
# model
non_linear_model = SVC(kernel='rbf')
# fit
non_linear_model.fit(X_train, y_train)
# predict
y_pred = non_linear_model.predict(X_val)

In [ ]:
# confusion matrix and accuracy
# accuracy
print("accuracy:", metrics.accuracy_score(y_true=y_val, y_pred=y_pred), "\n")
# cm
print(metrics.confusion_matrix(y_true=y_val, y_pred=y_pred))

In [ ]:
#creating a KFold object with 5 splits
folds = KFold(n_splits = 5, shuffle = True, random_state = 10)
# specify range of hyperparameters
# Set the parameters by cross-validation
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],
'C': [5,10]}]
# specify model
model = SVC(kernel="rbf")
# set up GridSearchCV()
model_cv = GridSearchCV(estimator = model,
param_grid = hyper_params,
scoring= 'accuracy',
cv = folds,
verbose = 1,
return_train_score=True)
# fit the model
model_cv.fit(X_train, y_train)

In [ ]:
# cv results
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
# converting C to numeric type for plotting on x-axis
cv_results['param_C'] = cv_results['param_C'].astype('int')
# # plotting
plt.figure(figsize=(16,8))
# subplot 1/3
plt.subplot(131)
gamma_01 = cv_results[cv_results['param_gamma']==0.01]
plt.plot(gamma_01["param_C"], gamma_01["mean_test_score"])
plt.plot(gamma_01["param_C"], gamma_01["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.01")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')
# subplot 2/3
plt.subplot(132)
gamma_001 = cv_results[cv_results['param_gamma']==0.001]
plt.plot(gamma_001["param_C"], gamma_001["mean_test_score"])
plt.plot(gamma_001["param_C"], gamma_001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')
# subplot 3/3
plt.subplot(133)
gamma_0001 = cv_results[cv_results['param_gamma']==0.0001]
plt.plot(gamma_0001["param_C"], gamma_0001["mean_test_score"])
plt.plot(gamma_0001["param_C"], gamma_0001["mean_train_score"])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.title("Gamma=0.0001")
plt.ylim([0.60, 1])
plt.legend(['test accuracy', 'train accuracy'], loc='upper left')
plt.xscale('log')

In [ ]:
# printing the optimal accuracy score and hyperparameters
best_score = model_cv.best_score_
best_hyperparams = model_cv.best_params_
print("The best test score is {0} corresponding to hyperparameters {1}".format(best_score, The best test score is 0.9448809523809524 corresponding to hyperparameters {'C': 10, 'gamma': 0.001}

In [ ]:
# model with optimal hyperparameters
# model
model = SVC(C=10, gamma=0.001, kernel="rbf")
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
# metrics
print("accuracy", metrics.accuracy_score(y_val, y_pred), "\n")
print(metrics.confusion_matrix(y_val, y_pred), "\n")